In [47]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy.linalg as npl
from scipy.optimize import minimize
import math as math
from itertools import combinations
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import WhiteKernel
import matplotlib.lines as mlines
from sklearn.model_selection import train_test_split
import csv
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import LinearSegmentedColormap
from scipy.interpolate import griddata
from scipy.spatial import distance_matrix
from scipy import interpolate
from sklearn.model_selection import KFold
from tqdm import tqdm

## Here I take 50 points from training data at random from each mass
I need to try where I take 200 points from whole data set and not 50 from each 

In [49]:
columns = ["x","y","z","w","data","error"]
datatab = pd.read_table("Updateddata.txt",names = columns)
datatab

,x,y,z,w,data,error
0,0.1,-1.439897,0.0,0.25,-2.831628,0.561478
1,0.1,-1.308997,0.5,0.25,-3.006489,0.485720
2,0.1,-1.178097,1.0,0.25,-2.354835,1.270193
3,0.1,-1.047198,1.0,0.25,-2.974002,0.983807
4,0.1,-0.916298,1.0,0.25,-2.401510,1.254378
...,...,...,...,...,...,...
995,0.7,1.570796,-1.0,1.00,-2.287634,2.337560
996,0.8,1.570796,-1.0,1.00,-2.164392,1.612457
997,0.9,1.570796,-1.0,1.00,-2.152368,1.550488
998,1.0,-1.439897,-1.0,1.00,-2.579505,0.442328


In [51]:
x = datatab['x']
y = datatab['y']
z = datatab['z']
w = datatab['w']
data = datatab['data']
error = datatab['error']

In [53]:
# First dataset 
datam1 = datatab[0:250]
xm1 = datam1['x'].values
ym1 = datam1['y'].values
zm1 = datam1['z'].values
wm1 = datam1['w'].values
dm1 = datam1['data'].values
em1 = datam1['error'].values
X1 = np.column_stack((xm1, ym1, zm1, wm1))

# Second dataset
datam2 = datatab[250:500]
xm2 = datam2['x'].values
ym2 = datam2['y'].values
zm2 = datam2['z'].values
wm2 = datam2['w'].values
dm2 = datam2['data'].values
em2 = datam2['error'].values
X2 = np.column_stack((xm2, ym2, zm2, wm2))

# Third dataset
datam3 = datatab[500:750]
xm3 = datam3['x'].values
ym3 = datam3['y'].values
zm3 = datam3['z'].values
wm3 = datam3['w'].values
dm3 = datam3['data'].values
em3 = datam3['error'].values
X3 = np.column_stack((xm3, ym3, zm3, wm3))

# Fourth dataset
datam4 = datatab[750:]
xm4 = datam4['x'].values
ym4 = datam4['y'].values
zm4 = datam4['z'].values
wm4 = datam4['w'].values
dm4 = datam4['data'].values
em4 = datam4['error'].values
X4 = np.column_stack((xm4, ym4, zm4, wm4))

In [126]:
X = np.column_stack((x,y, z, w))
D = np.concatenate((dm1,dm2,dm3,dm4))
E = np.concatenate((em1,em2,em3,em4))
print(D.shape)
print(E.shape)
print(X.shape)

(1000,)
(1000,)
(1000, 4)


In [216]:
n_splits = 2  

gpr_list = []

# Lists to store multiple iterations of splits
X_train_matrices, X_test_matrices = [], []
D_train_matrices, D_test_matrices = [], []
E_train_matrices, E_test_matrices = [], []
train_index_matrices, test_index_matrices = [], []

for split in tqdm(range(n_splits), desc="Training Progress"):
    # Perform train-test split for each dataset
    X1_train, X1_test, D1_train, D1_test, E1_train, E1_test, train1_index, test1_index = train_test_split(
        X1, dm1, em1, range(len(X1)), test_size=0.2, random_state=42 + split, shuffle=True)
    X2_train, X2_test, D2_train, D2_test, E2_train, E2_test, train2_index, test2_index = train_test_split(
        X2, dm2, em2, range(len(X2)), test_size=0.2, random_state=10 + split, shuffle=True)
    X3_train, X3_test, D3_train, D3_test, E3_train, E3_test, train3_index, test3_index = train_test_split(
        X3, dm3, em3, range(len(X3)), test_size=0.2, random_state=5 + split, shuffle=True)
    X4_train, X4_test, D4_train, D4_test, E4_train, E4_test, train4_index, test4_index = train_test_split(
        X4, dm4, em4, range(len(X4)), test_size=0.2, random_state=28 + split, shuffle=True)

    # Combine all training data
    X_train = np.concatenate((X1_train, X2_train, X3_train, X4_train))
    D_train = np.concatenate((D1_train, D2_train, D3_train, D4_train))
    
    # Define the Gaussian Process kernel
    bounds_l = ((.08, 10), (1e-2, 10.0), (1e-2, 10.0), (1e-2, 10.0))  
    guess_l = (1,1,1,1)
    guess_signal_var = 1.0
    bounds_signal_var = (1e-20, 1e20)

    kernel = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * \
             RBF(length_scale=guess_l, length_scale_bounds=bounds_l)
    
    # Train Gaussian Process Regressor
    gpr = GaussianProcessRegressor(kernel=kernel, alpha=E_train**2, normalize_y=True, n_restarts_optimizer=20)
    gpr.fit(X_train, D_train)
    gpr_list.append(gpr)

    # Append current split's train-test matrices for all datasets
    X_train_matrices.append(np.array([X1_train, X2_train, X3_train, X4_train]))
    X_test_matrices.append(np.array([X1_test, X2_test, X3_test, X4_test]))

    D_train_matrices.append(np.array([D1_train, D2_train, D3_train, D4_train]))
    D_test_matrices.append(np.array([D1_test, D2_test, D3_test, D4_test]))

    E_train_matrices.append(np.array([E1_train, E2_train, E3_train, E4_train]))
    E_test_matrices.append(np.array([E1_test, E2_test, E3_test, E4_test]))

    train_index_matrices.append(np.array([train1_index, train2_index, train3_index, train4_index]))
    test_index_matrices.append(np.array([test1_index, test2_index, test3_index, test4_index]))

    print(f"Split {split+1}: Fitted Kernel: {gpr.kernel_}")


Training Progress:  50%|█████     | 1/2 [00:20<00:20, 20.30s/it]

Split 1: Fitted Kernel: 0.816**2 * RBF(length_scale=[0.109, 0.431, 0.591, 2.24])


Training Progress: 100%|██████████| 2/2 [00:47<00:00, 23.67s/it]

Split 2: Fitted Kernel: 0.842**2 * RBF(length_scale=[0.187, 0.147, 1.07, 3.01])


In [224]:
gpr1 = gpr_list[0]
gpr2 = gpr_list[1]
Dfit1, Dstd1 = gpr1.predict(X1, return_std=True)
Dfit2, Dstd2 = gpr1.predict(X2, return_std=True)
Dfit3, Dstd3 = gpr1.predict(X3, return_std=True)
Dfit4, Dstd4 = gpr1.predict(X4, return_std=True)

In [226]:
test1 = test_index_matrices[0]
test1

array([[142,   6,  97,  60, 112, 181, 197, 184,   9, 104, 200, 216, 236,
        240,  67, 224, 194,  15, 177,  24, 247,  19, 114,  68, 176,  45,
         55,  10,  30, 120, 201,  33, 141, 219,  25, 111,  18, 196, 180,
        159, 118,  96, 108, 185, 222,  16, 146,  69, 173,  38],
       [ 56, 127,  60, 152, 106, 119, 194,  91, 216,  49, 163, 183, 240,
        212,  26, 224,  63, 147, 209,  87,  58, 238, 138, 144, 130,  55,
         10,  35, 232, 103, 136, 204,  80, 235, 186,   6, 166,  24,  76,
        105,  19, 114,  59, 124, 211,   1, 176, 187, 161,  61],
       [215, 173, 115, 185,  50,  81, 103, 108, 223,  74,  48,  56, 225,
        161, 130,  39, 191, 160, 150, 196, 141,  90,  82, 126, 167,  24,
         40, 151,  47, 187, 112, 197,  33,   0, 189,  41, 247, 123, 143,
        118, 202,   3, 163,   6, 200, 239,  88, 217,  25,  12],
       [142, 223, 191,  20, 132, 204, 217,  95, 161,  60, 117, 148, 202,
        172, 230, 207, 174,   1, 175, 197, 155, 229, 240,  92, 152, 134,
     

In [230]:
relative_difference_list = []
for j in range(n_splits):
    for i in range(4):
        Xtest =  X_test_matrices[j][i]
        x_coords = Xtest[:,0]
        y_coords = Xtest[:,1]
        testindices = test_index_matrices[j][i]
        Dtrue = D_test_matrices[j][i]
        gpr = gpr_list[j]
        Dpred = gpr.predict(Xtest)
        relative_difference = np.abs(Dtrue - Dpred) / np.abs(Dtrue)
        relative_difference_list.append(relative_difference)
        data = relative_difference
        mean_value = np.mean(data)
        std_value = np.std(data)
        max_value = np.max(data) 
        print(f"Split {j} Mass set {i} Results: Mean={mean_value:.5f},Standard Deviation={std_value:.5f},Maximum Value={max_value:.5f}")

Split 0 Mass set 0 Results: Mean=0.03341,Standard Deviation=0.03539,Maximum Value=0.17548
Split 0 Mass set 1 Results: Mean=0.02594,Standard Deviation=0.03100,Maximum Value=0.14816
Split 0 Mass set 2 Results: Mean=0.03422,Standard Deviation=0.03173,Maximum Value=0.14647
Split 0 Mass set 3 Results: Mean=0.03603,Standard Deviation=0.03254,Maximum Value=0.12787
Split 1 Mass set 0 Results: Mean=0.03534,Standard Deviation=0.02525,Maximum Value=0.10758
Split 1 Mass set 1 Results: Mean=0.04056,Standard Deviation=0.06824,Maximum Value=0.47853
Split 1 Mass set 2 Results: Mean=0.03325,Standard Deviation=0.06771,Maximum Value=0.49139
Split 1 Mass set 3 Results: Mean=0.03603,Standard Deviation=0.05916,Maximum Value=0.42789
